In [14]:
import torch
from torch import nn

from Generator import Generator
from Discriminator import ProjectedDiscriminator

In [ ]:
c_dim = 768
z_dim = 64
img_resolution = 128

generator     = Generator(z_dim = z_dim, conditional=True, img_resolution = img_resolution)
discriminator = ProjectedDiscriminator(c_dim = c_dim)

discriminator.name = "D"
generator.name     = "G"

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(


In [21]:
def partial_freeze(gen_or_disc: nn.Module) -> None:
    phase = gen_or_disc.name

    if phase == "G":

        trainable_layers = gen_or_disc.trainable_layers
        # Freeze all layers first
        gen_or_disc.requires_grad_(False)

        # Then selectively unfreeze based on substring match
        for name, layer in gen_or_disc.named_modules():
            should_train = any(layer_type in name for layer_type in trainable_layers)
            layer.requires_grad_(should_train)
    
    elif phase == "D":
        gen_or_disc.dino.requires_grad_(False)
    
    else: raise NotImplemented
    